In [0]:
#from analysis_helpers import load_learned_embeddings, create_filepaths
import pandas as pd 
import numpy as np
import librosa
import scipy
import statistics as stat
import pickle as pkl
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
%matplotlib inline

In [2]:
DATASET = 'Var'
SIZE = 128
SKIP_WINDOW = 1
QUANT_FACTOR = 1
embeddings_filepath, reverse_dictionary_filepath, dictionary_filepath, w2i_filepath, w2c_filepath = create_filepaths(DATASET,SIZE,SKIP_WINDOW,QUANT_FACTOR)
E, _, _, _, w2c = load_learned_embeddings(embeddings_filepath,reverse_dictionary_filepath,dictionary_filepath,w2i_filepath, w2c_filepath)
print(E.shape)

NameError: ignored

In [0]:
words,vocab = np.load('data/sizesQ1_frVar.npy');
print(words,vocab)

In [0]:
# pitch features

def PitchMean(contour):
    return stat.mean(contour)

def PitchSD(contour):
    return stat.pstdev(contour)

def DerivativeMean(contour):
    return stat.mean(np.diff(contour))

def DerivativeSD(contour):
    return stat.pstdev(np.diff(contour))

def Length(contour, frame_length=11.6e-3):
    return frame_length*len(contour)

def VibratoRate(contour, rate_median=6):
    #librosa.core.fft_frequencies(sr=(1/11.6e-3), n_fft=8)
    return librosa.segment.recurrence_matrix(contour)
def Lag(recmat):
    #librosa.core.fft_frequencies(sr=(1/11.6e-3), n_fft=8)
    return librosa.segment.recurrence_to_lag(recmat)

def autocorr(x):
    n = len(x)
    variance = x.var()
    x = x-x.mean()
    r = np.correlate(x, x, mode = 'full')[-n:]
    assert np.allclose(r, np.array([(x[:n-k]*x[-(n-k):]).sum() for k in range(n)]))
    result = r/(variance*(np.arange(n, 0, -1)))
    return result

def TremeloFreq(x, time_step=0.0116, nfft = 128):
    y = scipy.signal.detrend(x)
    #plt.plot(y)
    ps = (np.abs(np.fft.fftn(y,nfft))**2)/len(y)
    print(ps[:int(len(ps)/2)])
    freqs = librosa.core.fft_frequencies(sr=1/time_step, n_fft=nfft)
    freqs = freqs[1:]
    print(freqs)
    ps = ps[:int(len(ps)/2)]
    pks = scipy.signal.find_peaks(ps)
    fmax = freqs[np.argmax(ps)]
    print(fmax)
    plt.plot(freqs, ps)
    

def extractContourFeatures(x):
    mean = PitchMean(x)
    sd = PitchSD(x)
    dmean = DerivativeMean(x)
    dsd = DerivativeSD(x)
    length = Length(x)
    fdict = {'mean':mean, 'sd':sd, 'dmean':dmean, 'dsd':dsd, 'length':length}
    return fdict

In [0]:
section = 'Back'
contour_feature_dict = [extractContourFeatures(w2c[i][-15:]) for i in range(E.shape[0])]
outfile = 'data/contourFeatureDict{0}_Q{1}_frVar.pkl'.format(section,QUANT_FACTOR)
f = open(outfile,"wb")
pkl.dump(contour_feature_dict, f)
f.close()
print(section," saved")

section = 'Front'
contour_feature_dict = [extractContourFeatures(w2c[i][:15]) for i in range(E.shape[0])]
outfile = 'data/contourFeatureDict{0}_Q{1}_frVar.pkl'.format(section,QUANT_FACTOR)
f = open(outfile,"wb")
pkl.dump(contour_feature_dict, f)
f.close()
print(section," saved")

section = ''
contour_feature_dict = [extractContourFeatures(w2c[i]) for i in range(E.shape[0])]
outfile = 'data/contourFeatureDict{0}_Q{1}_frVar.pkl'.format(section,QUANT_FACTOR)
f = open(outfile,"wb")
pkl.dump(contour_feature_dict, f)
f.close()
print(section," saved")

In [0]:
contour_feature_dict[1]['length']

In [0]:
def plotContourFeatureDistribution(contour_feature_dict, rng):
    fig, axs = plt.subplots(2, 3, tight_layout=True, figsize=(15,5))
    axs = axs.flatten()
    # We can set the number of bins with the `bins` kwarg
    n_bins = 100
    #fig.suptitle("W2C Indices {0} - {1}".format(i_min,i_max), fontsize=20)
    axs[0].hist([contour_feature_dict[i]['mean'] for i in rng], bins=n_bins);
    axs[0].set_title('Mean')
    axs[1].hist([contour_feature_dict[i]['sd'] for i in rng], bins=n_bins);
    axs[1].set_title('SD')
    axs[2].hist([contour_feature_dict[i]['length'] for i in rng], bins=3);
    axs[2].set_xlim([0,6]); axs[2].set_title('Length')
    axs[3].hist([contour_feature_dict[i]['dmean'] for i in rng], bins=n_bins);
    axs[3].set_title('Deriv. Mean')
    axs[4].hist([contour_feature_dict[i]['dsd'] for i in rng], bins=n_bins);
    axs[4].set_title('Deriv. SD')

plotContourFeatureDistribution( contour_feature_dict, range(0, E.shape[0]) )